# PAndas preprocessing

In [ ]:
import pandas as pd

In [ ]:
df =  pd.read_html('https://en.wikipedia.org/wiki/List_of_chapters_in_the_Quran')
df = df[0]

In [ ]:
df.columns

In [ ]:
df2 = df[['Anglicized title(s)', 'Number of verses (Number of Rukūʿs)']]

In [ ]:
df2

In [ ]:
df2['Number of verses (Number of Rukūʿs)'] = df2['Number of verses (Number of Rukūʿs)'].str.split(' ', expand=True)[0]

In [ ]:
df2

In [ ]:
df2.to_csv('surah_aayah_count.csv', index=False)

# Scraping tests

In [1]:
import pandas as pd
import numpy as np
import requests
import os

In [2]:
surah_df = pd.read_csv('surah_aayah_count.csv').dropna()

def generate_dateset(imam):
    output_dir = 'data'
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    output_dir = os.path.join(output_dir, imam)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    for surah_index in range(0, 113):
        for aayah_number in range(1, int(surah_df.iloc[surah_index]['Number of verses (Number of Rukūʿs)'] + 1)):
            file_name = f'{str(surah_index+1).zfill(3)}{str(aayah_number).zfill(3)}'
            url = f"https://everyayah.com/data/{imam}/{file_name}.mp3"

            out_file_path = os.path.join(output_dir, f'{surah_index+1}_{aayah_number}.mp3')
            if os.path.exists(out_file_path):
                continue
            doc = requests.get(url)

            if doc.status_code == 200:
                print(f'{surah_index+1}_{aayah_number}.mp3')
                with open(out_file_path, 'wb') as f:
                    f.write(doc.content)
            else:
                raise Exception(f'Error: {doc.status_code} for {url}')

In [3]:
imam_df = pd.read_csv('audiocats.csv')
df_chunks = np.array_split(imam_df, 10)

In [5]:
df_chunks

[                                    imam
 0           Abdul_Basit_Mujawwad_128kbps
 1    AbdulSamad_64kbps_QuranExplorer.Com
 2           Abdul_Basit_Murattal_192kbps
 3  Abdullaah_3awwaad_Al-Juhaynee_128kbps
 4                Abdullah_Basfar_192kbps,
                                             imam
 5                       Abdullah_Matroud_128kbps
 6                 Abdurrahmaan_As-Sudais_192kbps
 7                  Abu_Bakr_Ash-Shaatree_128kbps
 8                            Ahmed_Neana_128kbps
 9  Ahmed_ibn_Ali_al-Ajamy_128kbps_ketaballah.net,
                           imam
 10     Akram_AlAlaqimy_128kbps
 11             Alafasy_128kbps
 12  Ali_Hajjaj_AlSuesy_128kbps
 13            Ali_Jaber_64kbps
 14         Ayman_Sowaid_64kbps,
                                         imam
 15  English/Sahih_Intnl_Ibrahim_Walk_192kbps
 16                        Fares_Abbad_64kbps
 17                            Ghamadi_40kbps
 18                        Hani_Rifai_192kbps
 19                    

In [6]:
def generate_dataset_parallel(df):
    for imam in df['imam'].unique():
        generate_dateset(imam)

In [7]:
from multiprocessing import Pool

with Pool(8) as p:
    print(p.map(generate_dataset_parallel, df_chunks))

: 

: 